We first start by importing the libraries and the data

In [1]:
import pandas as pd
from deap import base, creator, tools, algorithms
import random
import numpy as np

In [2]:
excel_file = pd.read_excel('Group Project.xlsx', sheet_name = None)
co2_emission = excel_file['CO2 Emission by Industry Sector']
rbf_equity = excel_file['RBF Equity Portfolio']

We now perform data extraction and preprocessing

In [3]:
prcntg_rtrns = rbf_equity['Investment Return in % (annual)']
percentage_returns = np.array(prcntg_rtrns.apply(lambda x: float(x.strip('%'))/100))
total_investment = rbf_equity['Investment in $million'].sum()
percentage_fossil = rbf_equity.iloc[3]['Investment in $million'] / total_investment
initial_allocation = rbf_equity['Investment in $million'] / total_investment
num_sectors = len(rbf_equity)

We now define functions to generate a portfolio, evaluate a given portfolio and ensure that all portfolio generated by our algorithm are valid, i.e. all entries are non negative and their sum is 1

In [4]:
def generate_allocation():  
    """Generate a new allocation array by adjusting the percentage of the fossil 
    sector and redistributing the difference proportionally among the other sectors.
    """
    allocations = np.array(initial_allocation.copy())
    new_fossil_percentage = random.uniform(0, percentage_fossil)
    difference = percentage_fossil - new_fossil_percentage
    fossil_index = 3  
    allocations[fossil_index] = new_fossil_percentage    
    total_redistribution = difference    
    other_sectors = [i for i in range(len(allocations)) if i != fossil_index]
    
    random_deltas = np.random.rand(len(other_sectors))
    random_deltas /= sum(random_deltas) 
    random_deltas *= total_redistribution      
    for i, delta in zip(other_sectors, random_deltas):
        allocations[i] += delta
    allocations /= np.sum(allocations)
    return allocations

In [5]:
def calculate_enb(allocation):
    """Calculate the Effective Number of Bets for a given allocation."""
    allocation = np.array(allocation)
    allocation = allocation[allocation > 0] / np.sum(allocation)
    enb = np.exp(-np.sum(allocation * np.log(allocation)))
    return enb

def calculate_hhi(allocation):
    """Calculate the Herfindahl-Hirschman Index for a given allocation."""
    return np.sum(np.square(allocation))
def calculate_volatility(allocation, sector_std):
    w = np.array(allocation)
    C = np.diag(sector_std ** 2)
    portfolio_volatility = np.sqrt(np.dot(w.T, np.dot(C, w)))
    return portfolio_volatility

def evalPortfolio(individual):
    """Evaluate the portfolio based on the weighted returns,
    CO2 emissions, and volatility."""
    fossil_fuel_index = 3
    weighted_volatility = calculate_volatility(individual,
                                               rbf_equity['Volatility (st. dev.)'])
    enb = calculate_enb(individual)
    hhi = calculate_hhi(individual)
    enb_penalty = np.exp(-enb)  
    hhi_penalty = hhi ** 4
    fossil_fuel_penalty = (individual[fossil_fuel_index] * 100) ** 2
    weighted_co2 = np.dot(co2_emission['Total CO2e in t/$1 million invested'],
                          individual)
    weighted_co2 *= 100
    return (-weighted_co2, -weighted_volatility, -enb_penalty, -hhi_penalty, -fossil_fuel_penalty)


In [6]:
def checkBounds(minimum, maximum):
    def decorator(func):
        def wrapper(*args, **kargs):
            fossil_fuel_index = 3
            green_energy_index = 2
            offspring = func(*args, **kargs)
            for child in offspring:
                for i in range(len(child)):
                    if i != fossil_fuel_index: 
                        min_bound = max(initial_allocation[i], minimum) 
                        max_bound = maximum - initial_allocation[fossil_fuel_index]
                    
                    else:
                        min_bound = minimum
                        max_bound = initial_allocation[fossil_fuel_index]
                    if i == green_energy_index:
                        child[i] *= 1.1
                        max_bound = 1.1
                    if child[i] < min_bound:
                        child[i] = min_bound
                    elif child[i] > max_bound:
                        child[i] = max_bound
                total = sum(child)
                if total != 1:
                    ## Normalize the child
                    child[:] = [x / total for x in child]
            return offspring
        return wrapper
    return decorator

We now set up our Genetic Algorithm

In [7]:
creator.create("FitnessMax", base.Fitness, weights=(0.2, 0.5, 1.5, 1.5, 3))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()

In [8]:
toolbox.register("attr_float", generate_allocation)
toolbox.register("individual", tools.initIterate, 
                 creator.Individual, toolbox.attr_float)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [9]:
toolbox.register("evaluate", evalPortfolio)
toolbox.register("mate", tools.cxTwoPoint) 
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.3, indpb=0.24)
toolbox.register("select", tools.selNSGA2)

In [10]:
toolbox.decorate("mate", checkBounds(0,1))
toolbox.decorate("mutate", checkBounds(0,1))

In [11]:
population = toolbox.population(n=100)
NGEN = 10000
CXPB = 0.6
MUTPB = 0.3
fitnesses = list(map(toolbox.evaluate, population))
for ind, fit in zip(population, fitnesses):
    ind.fitness.values = fit

We now start our genetic algorithm

In [12]:
for gen in range(NGEN):
    if (gen + 1) % 1000 == 0 and gen != 0: print(f"Starting generation: {gen + 1}")
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values
    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)

    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    population = offspring
    all_fitnesses = np.array([ind.fitness.values[0] for ind in population]) # Assuming we're focusing on the first objective
    fitness_variance = np.var(all_fitnesses)
    if fitness_variance < 1e-5: # Threshold for minimal change
        print(f"Stopping at generation {gen + 1} due to minimal change in fitness.")

Starting generation: 1000
Starting generation: 2000
Starting generation: 3000
Starting generation: 4000
Starting generation: 5000
Starting generation: 6000
Starting generation: 7000
Starting generation: 8000
Starting generation: 9000
Starting generation: 10000


We collect the best portfolio

In [13]:
best_ind = tools.selBest(population, 1)[0]

We display the best allocation for each investment and the metrics

In [14]:
print("Best portfolio allocation:")
for idx, sector in rbf_equity.iterrows():
    print(f"{sector['Sector']}: {best_ind[idx] * 100:.2f}%")
adjusted_returns = np.dot(percentage_returns, best_ind)
adjusted_co2 = np.dot(co2_emission['Total CO2e in t/$1 million invested'], best_ind)
adjusted_volatility = calculate_portfolio_volatility(best_ind, rbf_equity['Volatility (st. dev.)'])
adjusted_enb = calculate_enb(best_ind)
adjusted_hhi = calculate_hhi(best_ind)
print(f"Adjusted portfolio metrics:")
print(f"Returns: {adjusted_returns * 100:.2f}%")
print(f"CO2 emissions: {adjusted_co2:.2f}")
print(f"Volatility: {adjusted_volatility:.2f}")
print(f"Effective Number of Bets: {adjusted_enb:.2f}")
print(f"Herfindahl-Hirschman Index: {adjusted_hhi:.2f}")

Best portfolio allocation:
Consumer Discretionaries: 6.93%
Consumer Staples: 5.35%
Energy - Green energy: 3.39%
Energy - Fossil fuel: 0.00%
Financials: 40.86%
Healthcare: 24.18%
Industrials: 5.43%
Materials: 2.49%
Technology: 9.69%
Utilities: 1.68%
Adjusted portfolio metrics:
Returns: 11.59%
CO2 emissions: 487.15
Volatility: 8.94
Effective Number of Bets: 5.53
Herfindahl-Hirschman Index: 0.25
